Белокосов Максим 301 группа

In [ ]:
import csv
import requests
from datetime import datetime, timedelta, timezone
from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
REQUESTS_PER_MINUTE = 60
DELAY = 60 / REQUESTS_PER_MINUTE

In [ ]:
API_KEY = '07a06d9ae9124b88b15162244241610'
N = 390_000

In [ ]:
def fetch_current_weather(lat, lon):
    try:
        response = requests.get(
            f'https://api.weatherapi.com/v1/current.json?key={API_KEY}&q={lat},{lon}&aqi=no'
        )
        if response.status_code == 200:
            return response.json()
        else:
            print(f"Error fetching current weather for lat={lat}, lon={lon}: {response.status_code}")
            return None
    except Exception as e:
        print(f"Exception occurred fetching current weather: {e}")
        return None

In [ ]:
def fetch_historical_weather(lat, lon, date):
    try:
        formatted_date = datetime.fromtimestamp(date, tz=timezone.utc).strftime('%Y-%m-%d')
        url = f'http://api.weatherapi.com/v1/history.json?key={API_KEY}&q={lat},{lon}&dt={formatted_date}'
        response = requests.get(url)
        if response.status_code == 200:
            return response.json()
        else:
            print(f"Error fetching historical weather for lat={lat}, lon={lon}, date={formatted_date}: {response.status_code}")
            return None
    except Exception as e:
        print(f"Exception occurred fetching historical weather: {e}")
        return None

In [ ]:
df = pd.read_csv('translated_cities_with_coords.csv', encoding='utf-8')

In [ ]:
weather_condition = {'Overcast', 'Rain', 'Snow', 'Sunny', 'Clear', 'Partly cloudy', 'Blowing snow' , 'Thunderstorm', 'Patchy light drizzle' ,'Light rain', 'Heavy rain', 'Drizzle'}

In [ ]:
today = datetime.now(timezone.utc)

In [ ]:
df['current_temp'] = None
df['current_weather'] = None
df['historical_weather'] = None

In [ ]:
for i, row in df.iterrows():
        city_name = row['City']
        lat, lon = row['Latitude'], row['Longitude']

        # Текущая погода
        current_weather = fetch_current_weather(lat, lon)

        if current_weather:
            temp = current_weather['current']['temp_c']
            condition = current_weather['current']['condition']['text']
            df.at[i, 'current_temp'] = temp
            df.at[i, 'current_weather'] = condition

        # Историческая погода за 3 дня
        historical_weather_data = []
        for days_ago in range(3):
            date = int((today - timedelta(days=days_ago)).timestamp())
            historical_weather = fetch_historical_weather(lat, lon, date)

            if historical_weather:
                forecast_day = historical_weather['forecast']['forecastday'][0]
                historical_weather_data.append({
                    'date': forecast_day['date'],
                    'avg_temp': forecast_day['day']['avgtemp_c'],
                    'condition': forecast_day['day']['condition']['text']
                })

        # Сохраняем исторические данные в DataFrame
        df.at[i, 'historical_weather'] = historical_weather_data

In [ ]:
df_below_zero = df[df['current_temp'] < 0]
df_population_filtered = df[df['Population'] > N]
df_weather_filtered = df[df['current_weather'] == weather_condition]

In [ ]:
print("\nГорода, где температура ниже 0°C и население выше порога:")
print(df_below_zero[['City', 'Population', 'current_temp']])


Города, где температура ниже 0°C и население выше порога:
                  City  Population current_temp
0               Abakan      184769         -2.2
6              Angarsk      221296         -5.7
14             Achinsk      100621         -3.7
34              Bratsk      224071         -6.7
90         Zelenogorsk       54279         -2.3
97             Irkutsk      606369         -5.1
108              Kansk       86816         -4.3
110           Kemerovo      557119         -0.6
115          Kiselevsk       83431         -1.1
132        Krasnoyarsk     1205473         -2.0
146        Lesosibirsk       55730         -2.1
158     Mezhdurechensk       96174         -3.9
176          Neryungri       53409         -5.9
184       Novokuznetsk      537480         -0.5
196            Norilsk      174453         -9.3
217        Prokopyevsk      177819         -1.1
240            Seversk      106648         -0.8
268              Tomsk      556478         -0.9
274           Ulan-Ude      4

In [ ]:
print(f"\nГорода с текущей погодой {weather_condition} и населением выше порога:")
print(df_weather_filtered[['City', 'Population', 'current_weather']])


Города с текущей погодой {'Rain', 'Patchy light drizzle', 'Blowing snow', 'Thunderstorm', 'Sunny', 'Partly cloudy', 'Heavy rain', 'Snow', 'Overcast', 'Drizzle', 'Clear', 'Light rain'} и населением выше порога:
Empty DataFrame
Columns: [City, Population, current_weather]
Index: []


In [ ]:
print(f"\nИсторические данные по погоде:")
for i, row in df_population_filtered.iterrows():
        city_name = row['City']
        print(f"\nИсторическая погода для {city_name}:")
        for entry in row['historical_weather']:
            print(f"Дата: {entry['date']}, Средняя температура: {entry['avg_temp']}°C, Погода: {entry['condition']}")

        temperature_below_zero = all(entry['avg_temp'] < 0 for entry in row['historical_weather'])
        weather_condition_met = all(entry['condition'] == weather_condition for entry in row['historical_weather'])

        if temperature_below_zero:
            print(f"{city_name} - Температура ниже 0°C в течение трех дней подряд.")

        if weather_condition_met:
            print(f"{city_name} - Погода {weather_condition} в течение трех дней подряд.")



Исторические данные по погоде:

Историческая погода для Astrakhan:
Дата: 2024-10-17, Средняя температура: 12.2°C, Погода: Patchy rain possible
Дата: 2024-10-16, Средняя температура: 10.1°C, Погода: Light rain
Дата: 2024-10-15, Средняя температура: 9.0°C, Погода: Overcast

Историческая погода для Balashikha:
Дата: 2024-10-17, Средняя температура: 3.6°C, Погода: Sunny
Дата: 2024-10-16, Средняя температура: 6.2°C, Погода: Light rain shower
Дата: 2024-10-15, Средняя температура: 4.8°C, Погода: Light drizzle

Историческая погода для Barnaul:
Дата: 2024-10-17, Средняя температура: 0.8°C, Погода: Sunny
Дата: 2024-10-16, Средняя температура: -0.5°C, Погода: Partly cloudy
Дата: 2024-10-15, Средняя температура: 1.8°C, Погода: Sunny

Историческая погода для Vladivostok:
Дата: 2024-10-17, Средняя температура: 11.8°C, Погода: Partly cloudy
Дата: 2024-10-16, Средняя температура: 8.3°C, Погода: Sunny
Дата: 2024-10-15, Средняя температура: 14.6°C, Погода: Light rain shower

Историческая погода для Vo